# 사이킷런 모델

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [2]:
# iris datasets 로딩
iris = load_iris()

iris_data  = iris.data # feature
iris_label = iris.target # label

iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
iris_pdf = pd.DataFrame(iris_data, columns=iris_columns)
iris_pdf['target'] = iris_label
iris_pdf

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
iris_pdf.to_csv("./data/iris.csv", index=False)

## 1. 데이터 분할

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris_data,
    iris_label,
    test_size=0.2,
    random_state=42
)

## 2. 모델 정의

In [5]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf

DecisionTreeClassifier()

## 3. 훈련

In [6]:
# tree_clf 모델 자체를 훈련하는 것이므로 모델의 정보가 바뀜.
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

## 4. 예측

In [7]:
pred = tree_clf.predict(X_test)
print(pred)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


## 5. 성능평가

In [8]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred) # 정답과 예측 데이터 비교해 정확도 구하기
accuracy

1.0

# SparkML

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree-clf').getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 10:12:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
iris_filepath = "/home/ubuntu/working/spark-examples/data/iris.csv"

iris_sdf = spark.read.csv(f"file://{iris_filepath}", inferSchema=True, header=True)
iris_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|     0|
|         4.9|        3.0|         1.4|        0.2|     0|
|         4.7|        3.2|         1.3|        0.2|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
|         5.0|        3.6|         1.4|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



## 1. 데이터 분할

In [11]:
# DataFrame API의 randomSplit 메소드를 활용해 훈련 / 테스트 데이터 세트 분할

train_sdf, test_sdf = iris_sdf.randomSplit([0.8, 0.2], seed=42)

In [12]:
# 훈련 데이터 세트는 어떻게 변환되어도 하나만 존재하는 것이 좋음
# 모델 마다 공통적인 전처리를 모두 수행 후 train_sdf를 캐시에 넣어 놓기

train_sdf.cache()

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, target: int]

## 2. 행 벡터화

`VectorAssembler`를 이용해 모든 Feature에 해당하는 컬럼 생성

In [13]:
from pyspark.ml.feature import VectorAssembler

# 합쳐질 컬럼 목록
iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

In [14]:
# VectorAssembler로 지정한 컬럼을 하나의 행 벡터로 합치기

# inputCols : 합쳐질 컬럼'들'의 목록(list)
# outputCol : 합쳐진 새로운 컬럼의 이름
vec_assembler = VectorAssembler(inputCols=iris_columns, outputCol="features")

`Transformer` : 데이터프레임 변형 기능을 가진 클래스. dataframe 변형할 수 있는 `transform` method를 가짐.
- `VectorAssembler` : 데이터프레임을 변형하는 클래스
- `DecisionTreeClassifier` : Spark ML의 모델(Estimator)

In [15]:
# VectorAssembler Transform
train_feature_vector_sdf = vec_assembler.transform(train_sdf)
train_feature_vector_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.3|        3.0|         1.1|        0.1|     0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|     0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.2|         1.3|        0.2|     0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|     0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|     0|[4.6,3.1,1.5,0.2]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



## 3. 훈련
Estimator (추정기)
- Estimator란 예측할 수 있는 모델을 의미
- Spark ML의 모델(Estimator)은 데이터를 변환시키는 `Transformer`에 해당
    - Feature가 들어있는 데이터프레임을 받아 예측 값(prediction) 컬럼을 추가해주는 변환(Transformations)과정을 수행

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 껍데기만 생성 : 데이터 받아서 훈련 준비
# - 훈련데이터 컬럼의 feature, target을 미리 지정
dt = DecisionTreeClassifier(
    featuresCol = "features",
    labelCol = "target",
    maxDepth=5
)
type(dt)

pyspark.ml.classification.DecisionTreeClassifier

In [17]:
# 모델 학습
dt_model = dt.fit(train_feature_vector_sdf)
type(dt_model)

pyspark.ml.classification.DecisionTreeClassificationModel

## 4. 예측

In [18]:
test_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         4.4|        3.0|         1.3|        0.2|     0|
|         4.6|        3.2|         1.4|        0.2|     0|
|         4.6|        3.6|         1.0|        0.2|     0|
|         4.8|        3.1|         1.6|        0.2|     0|
|         4.9|        3.1|         1.5|        0.1|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



In [19]:
# 테스트 데이터 : 훈련데이터의 Transformer를 그대로 적용. 절대로 새롭게 만들지 않기.

test_feature_vector_sdf = vec_assembler.transform(test_sdf)
test_feature_vector_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



In [20]:
# 테스트 세트로 예측
predictions = dt_model.transform(test_feature_vector_sdf)
predictions.show(5)

+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
+------------+-----------+------------+-----------+------+------

- `rawPrediction` : 머신러닝 알고리즘에 의해 계산된 값.
    - 값에 대한 정확한 의미는 없음. 머신러닝 모델 알고리즘 마다 다를 수 있음
- `probability` : 예측 label 별 예측 확률 값
- `prediction` : 최종 예측 label 값

## 모델평가

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol = 'target',
    predictionCol = 'prediction',
    metricName = 'accuracy'
)

In [23]:
accuracy = evaluator_accuracy.evaluate(predictions)
accuracy

1.0

In [24]:
spark.stop()